# Extraer datos de WEB

Este Script se concentra en extraer la informacion y datos de la galeria del museo Van Gogh y las cartas escritas por el mismo artista para guardar los archivos de texto, png y otros datos necesarios para crear el set de datos de entrenamiento para un modelo de Machine Larning. Se utiliza BeautifulSoup para extraer la informacion desde los portales WEB de interes

## Objetivos

Extraer las imagenes, datos y metadatos de cada elemento dentro de la coleccion del museoVan Gogh.

## Fuente de Datos

Las fuentes de datos para el conjunto de datos son:

* Galeria de obras y trabajo de Van Gogh: https://vangoghmuseum.nl/en/search/collection?q=&artist=Vincent%20van%20Gogh
* Archivo de cartas escritas por Van Gogh: http://vangoghletters.org/vg/search/simple?term=
* Ruta de carpetas de recursos de las cartas escritas por Van Gogh: http://vangoghletters.org/vg/letters/


In [1]:
# importar librerias necesarias para scrapyWEB
import re
import os
import copy
import json
import urllib
import requests
import validators
from urllib.parse import urlparse
from bs4 import BeautifulSoup

# importar librerias para hacer un dataframe de referencia
import numpy as np
import pandas as pd

# import os
# import urllib.request, urllib.error, urllib.parse
# from os import remove
# import os.path
# from os import remove
# from numpy import array
# from bs4 import BeautifulSoup
# import requests
# import requests, pandas, numpy, matplotlib.pyplot
# from bs4 import BeautifulSoup
# import re
# import urllib
# import wget

## Galeria de Obras de vangoghmuseum.nl

### Procesamiento de las pinturas en la galeria

el proceso sigue los siguientes pasos, para facilidad de la estructura se propone un formato JSON por cada uno de los frames que se desean guardar en formato TXT, al final cada carpeta debe tener 4 archivos TXT mas una image en formato PNG. Los pasos a seguir son los siguientes:

* Chequeo que los enlaces existan y creo las carpetas necesarias.
* Extraigo las anotaciones de busqueda cada uno de los elementos de la galeria desde el objeto "Search in the collection:".
* Extraigo los datos archivisticos de cada elemento de la galeria desde el objeto "OBJECT DATA".
* Extraigo los trabajos relacionados si existen de cada elemento de la galeria desde el objeto "Related work".
* Extraigo la imagen en formato PNG de cada elemento de la galeria en el desde el objeto "DOWNLOAD IMAGE"


#### Enlaces utiles para el proceso

- Extract links from webpage (BeautifulSoup): https://pythonspot.com/extract-links-from-webpage-beautifulsoup/
- How to: Find all tags with some given name and attributes: https://kite.com/python/examples/1734/beautifulsoup-find-all-tags-with-some-given-name-and-attributes
- Beautiful Soup can't find the part of the HTML I want: https://stackoverflow.com/questions/51982930/beautiful-soup-cant-find-the-part-of-the-html-i-want
- Como chequear que un enlace esta vivo: https://stackoverflow.com/questions/51639585/checking-if-a-website-exist-with-python3
- Como utilizar libreria Request y el metodo GET: https://realpython.com/python-requests/
- Manejo de errores en Python 3 URL 1: https://www.python-course.eu/python3_exception_handling.php
- Manejo de errores en Python 3 URL 2: https://www.tutorialspoint.com/python3/python_exceptions.htm
- Descargar una imagen desde un URL: https://stackabuse.com/download-files-with-python/
- Reconocer si un string es un URL valido URL 1: https://stackoverflow.com/questions/7160737/python-how-to-validate-a-url-in-python-malformed-or-not
- Reconocer si un string es un URL valido URL 2: https://www.codespeedy.com/check-if-a-string-is-a-valid-url-or-not-in-python/
- Reconocer si un string tiene un substring importante URL: https://stackoverflow.com/questions/50432978/find-substring-of-keyword-using-beautifulsoup
- Remplazar signos de puntuacion entre palabras por espacios URL: https://stackoverflow.com/questions/44263446/python-regex-to-add-space-after-dot-or-comma
- Como guardar un archivo JSON en Python: https://stackoverflow.com/questions/12309269/how-do-i-write-json-data-to-a-file
- Recomendaciones de uso del JSON en Python: https://stackabuse.com/reading-and-writing-json-to-a-file-in-python/

### Definicion del DataFrame para la galeria
defino las columnas necesarias del DataFrame para saber que he podido recuperar y que no, TRUE es que si lo tengo, FALSE de lo contrario

In [2]:
# lista de columnas para trabajar el DataFrame de la galeria, sirve para manejar la calidad del proceso
# conocer los errores y ver que datos faltan por cada elemento de la galeria
soupCol = [
    "ID", # identificador unico de la galeria, tambien es el nombre del folder dentro del directorio local
    "NAME", # nombre del elemento en la galeria
    "ELEMENT_URL", # enlace del elemento recuperado del ScrapyWEB
    "DOWNLOAD_URL", # enlace de la imagen dentro del elemento de la galeria

    "HAS_ID", # booleano que identifica si se tiene un folder local del elemento
    "HAS_NAME", # booleano que identifica si se tiene el nombre del elemento
    
    "HAS_DESCRIPTION", # booleano que identifica si se tiene la seccion de descripcion en el HTML del elemento
    "HAS_DOWNLOAD", # booleano que identifica si se tiene la seccion de enlace de descarga en el HTML del elemento
    "HAS_TAGS", # booleano que identifica si se tiene la seccion de tags de busqueda en el HTML del elemento
    "HAS_DATA", # booleano que identifica si se tiene la seccion de datos de archivo en el HTML del elemento
    "HAS_RELATEDW", # booleano que identifica si se tiene la seccion de trabajo relacionado en el HTML del elemento
    
    "ERR_ID", # si no se puede crear la carpeta con el ID se guarda el error aca
    "ERR_NAME", # si no se obtiene el nombre se guarda el error aca
    
    "ERR_DESCRIPTION", # si no se obtiene la descripcion se guarda el error aca 
    "ERR_DOWNLOAD", # si no se obtiene el enlace de descarga se guarda el error aca 
    "ERR_TAGS", # si no se obtiene los tags de busqueda se guarda el error aca 
    "ERR_DATA", # si no se obtiene los datos de archivo se guarda el error aca 
    "ERR_RELATEDW", # si no se obtiene el trabajo relacionado se guarda el error aca 
    
    "DESCRIPTION", # aqui guardo el JSON con la informacion de la descripcion
    "TAGS", # aqui guardo el JSON con la informacion de los tags de busqueda
    "DATA", # aqui guardo el JSON con la informacion de los datos de coleccion
    "RELATEDW", # aqui guardo el JSON con la informacion del trabajo relacionado con cada una de las obras
]

In [3]:
# creo el dataFrame vacio con las columnas necesarias
paintsFrame = pd.DataFrame(columns = soupCol)

# si existe el CSV se carga lo que existe
galleryFile = "VanGoghGalleryIndex.csv"
dataFolder = "data"

if os.path.exists(dataFolder):
    
    # si el CSV de index ya existe
    galleryBackup = os.path.join(os.getcwd(), dataFolder, galleryFile)
    
    if os.path.exists(galleryBackup):
        
        # lo leo y lo cargo en el DataFrame
        paintsFrame = pd.read_csv(galleryBackup, sep = ",", encoding = "utf-8", engine = "python")#, index_col = [0])

In [4]:
# pruebo si todo cargo bien
paintsFrame.head()

,ID,NAME,ELEMENT_URL,DOWNLOAD_URL,HAS_ID,HAS_NAME,HAS_DESCRIPTION,HAS_DOWNLOAD,HAS_TAGS,HAS_DATA,...,ERR_NAME,ERR_DESCRIPTION,ERR_DOWNLOAD,ERR_TAGS,ERR_DATA,ERR_RELATEDW,DESCRIPTION,TAGS,DATA,RELATEDW
0,s0005V1962,"The Potato Eaters, 1885",https://vangoghmuseum.nl/en/collection/s0005V1962,https://vangoghmuseum.nl/download/8bc21467-2e7...,True,True,True,True,True,N.A.,...,NaN,NaN,NaN,NaN,N.A.,N.A.,"{\n ""tittle"": ""The Potato Eaters"",\n ""au...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
1,s0019V1962,Garden with Courting Couples: Square Saint-Pie...,https://vangoghmuseum.nl/en/collection/s0019V1962,https://vangoghmuseum.nl/download/12bd07b2-3c2...,True,True,True,True,True,N.A.,...,NaN,NaN,NaN,NaN,N.A.,N.A.,"{\n ""tittle"": ""Garden with Courting Couples...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
2,s0027V1962,"The Langlois Bridge, 1888",https://vangoghmuseum.nl/en/collection/s0027V1962,https://vangoghmuseum.nl/download/c82c2fcd-90e...,True,True,True,True,True,N.A.,...,NaN,NaN,NaN,NaN,N.A.,N.A.,"{\n ""tittle"": ""The Langlois Bridge"",\n ""...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
3,s0029V1962,"The Sower, 1888",https://vangoghmuseum.nl/en/collection/s0029V1962,https://vangoghmuseum.nl/download/61fbedad-1d6...,True,True,True,True,True,N.A.,...,NaN,NaN,NaN,NaN,N.A.,N.A.,"{\n ""tittle"": ""The Sower"",\n ""authorship...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
4,s0031V1962,"Sunflowers, 1889",https://vangoghmuseum.nl/en/collection/s0031V1962,https://vangoghmuseum.nl/download/597e7189-79d...,True,True,True,True,True,N.A.,...,NaN,NaN,NaN,NaN,N.A.,N.A.,"{\n ""tittle"": ""Sunflowers"",\n ""authorshi...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN


### Pagina Fuente (Root Page)
desde la pagina fuente recupero todos los enlaces a las obras de Vangohg y despues con la lista de enlaces recupero la informacion necesaria

#### Creando el indice de la coleccion
creo el URL de busqueda en la pagina principal de la coleccion para extraer los URLs de los elementos creados por Van Gogh, para pruebas mantengo un numero limitado de URLs a recuperar.

In [5]:
# Se recorre la pagina principal y recuperan los enlaces de l 
# numero de paginas/obras/enlaces a iterar

paints = 20
defaultPaints = 50

# maximo numero de pinturas conocidas
maxPaints = 1443

# busqueda base en la coleccion de Vangogh
paintsSearch = "https://vangoghmuseum.nl/en/search/collection?q=&artist=Vincent%20van%20Gogh&pagesize="
paintsRoot = "https://vangoghmuseum.nl"

# por defecto pruebo 50 elementos de la coleccion
paintsPage = paintsSearch + str(defaultPaints)

# si el numero de enlaces a buscar esta en el rango a propiado
if paints <= maxPaints and paints > 0:
    paintsPage = paintsSearch + str(paints)

# si el numero de enlaces es superior al maximo
elif paints > maxPaints:
    paintsPage = paintsSearch + str(maxPaints)

# si hay algo raro
else:
    paintsPage = paintsSearch + str(defaultPaints)

# reviso que cargue bien la pagina base
print("--- Search URL ---")
print(paintsPage)

--- Search URL ---
https://vangoghmuseum.nl/en/search/collection?q=&artist=Vincent%20van%20Gogh&pagesize=20


In [37]:
# lista de enlaces, IDs nombres a elementos de la coleccion
links = list()
ids = list()
names = list()

# objetos necesarios para scrapy en beatifulsoup: handler y resultados
soup = None
linkSoup = list()
nameSoup = list()
errIDs = list()

# diccionario caracterisico para la busqueda para el enlace del elemento de la galeria
linkAttr = {"class":"link-teaser triggers-slideshow-effect", "href":re.compile("^/en/collection/")}
# diccionario caracterisico para la busqueda del nombre del elemento de la galeria
nameAttr = {"class":"text-base text-dark"}

try:
    # GET del URL
    pageReq = requests.get(paintsPage)

    # si el GET me responde bien con codigo 200
    if pageReq.status_code == 200:

        # uso beautifulSoup
        soup = BeautifulSoup(pageReq.content, "html.parser")

        # esta es la seccion del HTML donde estan los enlaces con tag 'a'
        linkSoup = soup.body.findAll("a", attrs = linkAttr)
        
        # esta es la seccion del HTML donde estan los enlaces con tag 'h3'
        nameSoup = soup.body.findAll("h3", attrs = nameAttr)
        
        # si no hay errores agrego un mensaje vacio para no danhar el dataframe
        errIDs.append(None)

# si algo pasa aca esto esta muy mal
except Exception as e:
    print("Error in request: " + str(e))
    errIDs.append(e)
    print("Status Code: " + str(pageReq.status_code))

In [38]:
# saco los links e IDs a las paginas que quiero dentro de la coleccion
# estan los elementos de tipo link-teaser y la expresion regular del href
names = list()
errNames = list()
hasNames = list()

for name in nameSoup:

    try:

        # reconstruyo el enlace completo
        tempName = re.sub("\s+", " ", name.string)
        tempName = tempName[1:-1]
        names.append(tempName)
        
        # si no hay errores agrego un mensaje vacio para no danhar el dataframe
        errNames.append(None)
        hasNames.append(bool(True))

    except Exception as e:
        print("Error in: " + str(e))
        errNames.append(e)
        hasNames.append(bool(False))

In [39]:
# saco los links e IDs a las paginas que quiero dentro de la coleccion
# estan los elementos de tipo link-teaser y la expresion regular del href
ids = list()
links = list()

# lista de mensjaes de error y de booleanos de la operacion de sobre IDs
errIDs = list()
hasIDs = list()

for link in linkSoup:

    try:

        # reconstruyo el enlace completo
        tempLink = urllib.parse.urljoin(paintsRoot, link.get("href"))
        tempID = link.get("href").replace("/en/collection/", "")
        
        # nombre e ID para la columna del dataFrame
        ids.append(tempID)
        links.append(tempLink)
        
        # si no hay errores agrego un mensaje vacio para no danhar el dataframe
        errIDs.append(None)
        hasIDs.append(bool(True))

    except Exception as e:
        print("Error in: " + str(e))
        errIDs.append(e)
        hasIDs.append(bool(False))


In [40]:
# agregos los detalles del scrapy en las columnas del dataframe de calidad (nombres, IDs y URLs)
paintsFrame["ID"] = ids
paintsFrame["ELEMENT_URL"] = links 
paintsFrame["NAME"] = names
paintsFrame["ERR_ID"] = errIDs
paintsFrame["HAS_ID"] = hasIDs
paintsFrame["ERR_NAME"] = errNames
paintsFrame["HAS_NAME"] = hasNames

In [42]:
#chequeo la informacion recuperada de la busqueda
print("Extracted links in search: " + str(len(links)))
print("Repeated link elements: " + str(len(links) != len(set(links))))

print("Extracted IDs in search: " + str(len(ids)))
print("Repeated IDs elements: " + str(len(ids) != len(set(ids))))

print("Extracted names in search: " + str(len(names)))
print("Repeated names elements: " + str(len(names) != len(set(names))))

print("Shape: " + str(paintsFrame.shape))

paintsFrame.info()
paintsFrame.head()

Extracted links in search: 20
Repeated link elements: False
Extracted IDs in search: 20
Repeated IDs elements: False
Extracted names in search: 20
Repeated names elements: False
Shape: (20, 22)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 22 columns):
ID                 20 non-null object
NAME               20 non-null object
ELEMENT_URL        20 non-null object
DOWNLOAD_URL       20 non-null object
HAS_ID             20 non-null bool
HAS_NAME           20 non-null bool
HAS_DESCRIPTION    20 non-null bool
HAS_DOWNLOAD       20 non-null bool
HAS_TAGS           20 non-null bool
HAS_DATA           20 non-null object
HAS_RELATEDW       20 non-null object
ERR_ID             0 non-null object
ERR_NAME           0 non-null object
ERR_DESCRIPTION    0 non-null object
ERR_DOWNLOAD       0 non-null object
ERR_TAGS           0 non-null object
ERR_DATA           20 non-null object
ERR_RELATEDW       20 non-null object
DESCRIPTION        20 non-null obj

,ID,NAME,ELEMENT_URL,DOWNLOAD_URL,HAS_ID,HAS_NAME,HAS_DESCRIPTION,HAS_DOWNLOAD,HAS_TAGS,HAS_DATA,...,ERR_NAME,ERR_DESCRIPTION,ERR_DOWNLOAD,ERR_TAGS,ERR_DATA,ERR_RELATEDW,DESCRIPTION,TAGS,DATA,RELATEDW
0,s0005V1962,"The Potato Eaters, 1885",https://vangoghmuseum.nl/en/collection/s0005V1962,https://vangoghmuseum.nl/download/8bc21467-2e7...,True,True,True,True,True,N.A.,...,None,None,None,None,N.A.,N.A.,"{\n ""tittle"": ""The Potato Eaters"",\n ""au...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
1,s0019V1962,Garden with Courting Couples: Square Saint-Pie...,https://vangoghmuseum.nl/en/collection/s0019V1962,https://vangoghmuseum.nl/download/12bd07b2-3c2...,True,True,True,True,True,N.A.,...,None,None,None,None,N.A.,N.A.,"{\n ""tittle"": ""Garden with Courting Couples...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
2,s0027V1962,"The Langlois Bridge, 1888",https://vangoghmuseum.nl/en/collection/s0027V1962,https://vangoghmuseum.nl/download/c82c2fcd-90e...,True,True,True,True,True,N.A.,...,None,None,None,None,N.A.,N.A.,"{\n ""tittle"": ""The Langlois Bridge"",\n ""...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
3,s0029V1962,"The Sower, 1888",https://vangoghmuseum.nl/en/collection/s0029V1962,https://vangoghmuseum.nl/download/61fbedad-1d6...,True,True,True,True,True,N.A.,...,None,None,None,None,N.A.,N.A.,"{\n ""tittle"": ""The Sower"",\n ""authorship...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
4,s0031V1962,"Sunflowers, 1889",https://vangoghmuseum.nl/en/collection/s0031V1962,https://vangoghmuseum.nl/download/597e7189-79d...,True,True,True,True,True,N.A.,...,None,None,None,None,N.A.,N.A.,"{\n ""tittle"": ""Sunflowers"",\n ""authorshi...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN


In [43]:
# guardando los links en TXT y CSV para backup del proceso y control de calidad
galleryIndex = "VanGoghGalleryIndex.txt"
galleryFile = "VanGoghGalleryIndex.csv"
dataFolder = "data"

# guardando archivo original de texto
# si no existe el directorio
if not os.path.exists(dataFolder):
    
    os.makedirs(dataFolder)

# si existe el directorio
else: 
    
    # sobreescribo el archivo siempre
    # guardo los enlaces en un TXT por si acaso
    with open(os.path.join(os.getcwd(), dataFolder, galleryIndex), "w", encoding = "utf-8", errors = "ignore") as file:
        for link in links:
            file.write("%s\n" % link)
    
    # guardo el DataFrame segun las ultimas actualizaciones
    paintsFrame.to_csv(os.path.join(os.getcwd(), dataFolder, galleryFile), sep = ",", index = False, encoding = "utf-8", mode = "w")

In [12]:
#chquueo como va todo
paintsFrame.head()

,ID,NAME,ELEMENT_URL,DOWNLOAD_URL,HAS_ID,HAS_NAME,HAS_DESCRIPTION,HAS_DOWNLOAD,HAS_TAGS,HAS_DATA,...,ERR_NAME,ERR_DESCRIPTION,ERR_DOWNLOAD,ERR_TAGS,ERR_DATA,ERR_RELATEDW,DESCRIPTION,TAGS,DATA,RELATEDW
0,s0005V1962,"The Potato Eaters, 1885",https://vangoghmuseum.nl/en/collection/s0005V1962,https://vangoghmuseum.nl/download/8bc21467-2e7...,True,True,True,True,True,N.A.,...,,NaN,NaN,NaN,N.A.,N.A.,"{\n ""tittle"": ""The Potato Eaters"",\n ""au...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
1,s0019V1962,Garden with Courting Couples: Square Saint-Pie...,https://vangoghmuseum.nl/en/collection/s0019V1962,https://vangoghmuseum.nl/download/12bd07b2-3c2...,True,True,True,True,True,N.A.,...,,NaN,NaN,NaN,N.A.,N.A.,"{\n ""tittle"": ""Garden with Courting Couples...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
2,s0027V1962,"The Langlois Bridge, 1888",https://vangoghmuseum.nl/en/collection/s0027V1962,https://vangoghmuseum.nl/download/c82c2fcd-90e...,True,True,True,True,True,N.A.,...,,NaN,NaN,NaN,N.A.,N.A.,"{\n ""tittle"": ""The Langlois Bridge"",\n ""...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
3,s0029V1962,"The Sower, 1888",https://vangoghmuseum.nl/en/collection/s0029V1962,https://vangoghmuseum.nl/download/61fbedad-1d6...,True,True,True,True,True,N.A.,...,,NaN,NaN,NaN,N.A.,N.A.,"{\n ""tittle"": ""The Sower"",\n ""authorship...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
4,s0031V1962,"Sunflowers, 1889",https://vangoghmuseum.nl/en/collection/s0031V1962,https://vangoghmuseum.nl/download/597e7189-79d...,True,True,True,True,True,N.A.,...,,NaN,NaN,NaN,N.A.,N.A.,"{\n ""tittle"": ""Sunflowers"",\n ""authorshi...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN


### Directorio y Carpetas para persistencia
configuro donde se va a guardar la informacion inicial dentro de OneDrive (gitHub no me deja), creo las carpetas y desacoplo el computador donde corre el script. Despues ejecuto los siguientes pasos:

- Creo el folder fuente "01 - Raw".
- Dentro del folder fuente creo los folderes "01 - Paints" y "02 - Letters".
- Creo las carpetas necesarias dentro de "01 - Paints" con el ID del dataframe.

In [13]:
# defino los directorios locales donde se va a persistir la informacion
# no olvidarse del cambiar "\" a "\\" en los filepath de windows por que si no no sirve nada
# dir de santiago
SFAM_ROOT = "C:\\Users\\Felipe\\OneDrive - Universidad de Los Andes\\03 - PhD\\04 - Clases\\05 - IA en Arte y Disenho\\03 - Proyecto\\01 - Data\\01 - Raw"
# dir de daniela
DCP_ROOT = ""
# variable intermedia para independizar, se comenta uno u otro segun donde se corra
WORK_ROOT = SFAM_ROOT
# WORK_ROOT = DCP_ROOT

# nombres de carpetas utiles donde se guarda la informacion
rawFolder = "01 - Raw"
paintsFolder = "01 - Paints"
lettersFolder = "02 - Letters"

#La ruta se obtiene con el dirpath y el filename y el dirpath.split(os.path.sep)[-1] agrega la clase
DATA_ROOT = os.path.dirname(WORK_ROOT)
print(DATA_ROOT)

C:\Users\Felipe\OneDrive - Universidad de Los Andes\03 - PhD\04 - Clases\05 - IA en Arte y Disenho\03 - Proyecto\01 - Data


In [14]:
# preparar el directorio de trabajo
# si la ruta de trabajo local esta definida
workPath = ""

# si no eexiste el ROOT apropiado, esto lo debe arreglar el usuario
if not os.path.exists(WORK_ROOT):
    
    print("Make sure the WORK_ROOT is correct in your local HD!!!...")

# si si existe el root apropiado, configuro los folderes para guardar la informacion
elif os.path.exists(WORK_ROOT):

    # concateno los folders para la nueva carpeta de trabajo
    workPath = os.path.join(DATA_ROOT, rawFolder, paintsFolder)

    # si todo va bien y necesito crear nuevo folder de trabajo
    if not os.path.exists(workPath):
        
        # creo folderes nuevos
        os.makedirs(workPath)
        print("creating proper folder configuration!!!...")
    
    # si los folderes ya existen
    if os.path.exists(workPath):
        
        print("Working folders and config are cool... carry on!...")

Working folders and config are cool... carry on!...


### Crear subfolders de las pinturas
Con la columna de "ELEMENT_URL" e "ID" del data frame se hacen los siguientes pasos:

- Revisar los enlaces de "ELEMENT_URL" para ver si se puede recuperar la informacion de la galeria.
- crear folders por cada elemento de la galeria con nombre basado en el "ID".
- si ya estan las cosas no se repite el trabajo.

In [15]:
# recorro el arreglo de enlaces habilitados para extraer la informacion
# creo las carpetas necesarias, si ya existen no las sobreescribo.
links = list(paintsFrame["ELEMENT_URL"])
# en este caso los IDs son los nombres de los folders en local
folders = list(paintsFrame["ID"])

# variables de control de calidad del proceso, en este caso en las columnas, "ID", "HAS_ID", "ERR_ID"
errList = list()
hasList = list()

#inicio del ciclo para los enlaces, zip funciona para iterar 2 arreglos al tiempo
for link, folder in zip(links, folders):
            
    # chequeo si el enlace sirve para sacar la informacion
    try:
        # GET del URL
        linkReq = requests.get(link)
        
        # si el GET me responde bien con codigo 200
        if linkReq.status_code == 200:
                                
            # creo path para cada elemento en la coleccion
            tempPaintFolder = os.path.join(workPath, folder)

            # creo la carpeta del nuevo elemento de la coleccion si no existe
            if not os.path.exists(tempPaintFolder):

                os.makedirs(tempPaintFolder)

            # si existe la carpeta no hago nada
            if os.path.exists(tempPaintFolder):
                # print("Folder " + str(folder) + " already exists...")
                pass
        
            # si no hay errores agrego un mensaje vacio para no danhar el dataframe
            errList.append(None)
            hasList.append(bool(True))

    except Exception as e:
        print("Error in: " + str(e))
        print("Status Code: " + str(linkReq.status_code))
        errList.append(e)
        hasList.append(bool(False))

In [16]:
# actualizo el dataFrame para ver si hay errores
paintsFrame["HAS_ID"] = hasList
paintsFrame["ERR_ID"] = errList

In [17]:
# chequeo como va el dataframe de procesamiento
paintsFrame.info()
paintsFrame.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 22 columns):
ID                 20 non-null object
NAME               20 non-null object
ELEMENT_URL        20 non-null object
DOWNLOAD_URL       20 non-null object
HAS_ID             20 non-null bool
HAS_NAME           20 non-null bool
HAS_DESCRIPTION    20 non-null bool
HAS_DOWNLOAD       20 non-null bool
HAS_TAGS           20 non-null bool
HAS_DATA           20 non-null object
HAS_RELATEDW       20 non-null object
ERR_ID             0 non-null object
ERR_NAME           20 non-null object
ERR_DESCRIPTION    0 non-null float64
ERR_DOWNLOAD       0 non-null float64
ERR_TAGS           0 non-null float64
ERR_DATA           20 non-null object
ERR_RELATEDW       20 non-null object
DESCRIPTION        20 non-null object
TAGS               20 non-null object
DATA               0 non-null float64
RELATEDW           0 non-null float64
dtypes: bool(5), float64(5), object(12)
memory usage: 2.9+ KB


,ERR_DESCRIPTION,ERR_DOWNLOAD,ERR_TAGS,DATA,RELATEDW
count,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN


In [18]:
# guardando los links en TXT y CSV para backup del proceso y control de calidad
galleryFile = "VanGoghGalleryIndex.csv"
dataFolder = "data"

# actualizando archivo original CSV
# si no existe el directorio
if not os.path.exists(dataFolder):
    
    os.makedirs(dataFolder)

# si existe el directorio
else: 
    
    # guardo el DataFrame segun las ultimas actualizaciones
    paintsFrame.to_csv(os.path.join(os.getcwd(), dataFolder, galleryFile), sep = ",", index = False, encoding = "utf-8", mode = "w")

### Recuperar enlace de imagen de la galeria
Con la columna de "ELEMENT_URL", "ID" y "DOWNLOAD_URL" del dataFrame se hacen los siguientes pasos:

- Revisar si existe un enlace valido en "DOWNLOAD_URL", si no existe se sigue adelante.
- Si no existe el enlace valido, utilizo
Revisar los enlaces de "ELEMENT_URL" para ver si se puede recuperar la informacion de la galeria.
- crear folders por cada elemento de la galeria con nombre basado en el "ID".
- si ya estan las cosas no se repite el trabajo.

In [19]:
# recorro el arreglo de enlaces habilitados para extraer la informacion
# creo las carpetas necesarias, si ya existen no las sobreescribo.
links = list(paintsFrame["ELEMENT_URL"])
# en este caso los IDs son los nombres de los folders en local
folders = list(paintsFrame["ID"])
# lista de enlaces de descarga de imagenes en la galeria
downloads = list(paintsFrame["DOWNLOAD_URL"])
# lista de posibles nuevos enlaces de imagenes en la galeria
newloads = list()

# variables de control de calidad del proceso, en este caso en las columnas, "ID", "HAS_ID", "ERR_ID"
errList = list()
hasList = list()

# atributos de la busqueda en beatifulsoup
downAttrs = {"class":"button dark-hover", "href":re.compile("^/download/")}

#inicio del ciclo para los enlaces, zip funciona para iterar 2 arreglos al tiempo
for link, folder, down in zip(links, folders, downloads):
    
    # chequeo si el enlace sirve para sacar la informacion
    try:
        # inicializo uuna variable temporal
        tempLink = ""
        
        # chequeo si existe un enlace de descarga de imagen y uno del elemento de la galeria
        if not validators.url(str(down)) and validators.url(link):
            
            # GET del URL
            linkReq = requests.get(link)
        
            # si el GET me responde bien con codigo 200
            if linkReq.status_code == 200:
         
                # creo path para cada elemento en la coleccion
                tempPaintFolder = os.path.join(workPath, folder)

                # si existe la carpeta, descargo la imagen de la obra dentro de la coleccion de una vez
                if os.path.exists(tempPaintFolder):

                    # parse del cuerpo del elemento de la coleccion
                    linkSoup = BeautifulSoup(linkReq.content, "html.parser")

                    # busco todos los elementos de tipo class="button dark-hover"
                    downloadSoup = linkSoup.find("a", attrs = downAttrs)

                    # creo el enlace para descargar la imagen
                    tempLink = urllib.parse.urljoin(paintsRoot, downloadSoup.get("href"))

                    # pido el enlace de la imagen
                    downReq = requests.get(tempLink)

                    # creo el nombre y la direccion del archivo que quiero guardar en la carpeta
                    fileName = urlparse(tempLink)
                    fileName = fileName.path.split("/")[len(fileName.path.split("/"))-1]
                    filePath = os.path.join(tempPaintFolder, fileName)
                    
                    # si el archivo no existe lo guardo
                    if not os.path.exists(filePath):

                        # creo un archivo jpg para guardar la imagen del enlace
                        with open(filePath, "wb") as file:
                            file.write(downReq.content)
                            file.close()
                            
                    # no importa lo que pase guardo el link de descarga 
                    newloads.append(tempLink)
                        
        # si ya existe el URL de descarga lo repito en la columna
        if validators.url(str(down)) and validators.url(link):
            
            newloads.append(down)

        # si no hay errores agrego un mensaje vacio para no danhar el dataframe
        errList.append(None)
        hasList.append(bool(True))

    except Exception as e:
        print("Error in: " + str(e))
        print("Status Code: " + str(linkReq.status_code))
        newloads.append(tempLink)
        errList.append(e)
        hasList.append(bool(False)) 

In [20]:
# actualizo el dataFrame para ver si hay errores
paintsFrame["DOWNLOAD_URL"] = newloads
paintsFrame["HAS_DOWNLOAD"] = hasList
paintsFrame["ERR_DOWNLOAD"] = errList

In [33]:
# chequeo como va el dataframe de procesamiento
paintsFrame.info()
paintsFrame.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 22 columns):
ID                 20 non-null object
NAME               20 non-null object
ELEMENT_URL        20 non-null object
DOWNLOAD_URL       20 non-null object
HAS_ID             20 non-null bool
HAS_NAME           20 non-null bool
HAS_DESCRIPTION    20 non-null bool
HAS_DOWNLOAD       20 non-null bool
HAS_TAGS           20 non-null bool
HAS_DATA           20 non-null object
HAS_RELATEDW       20 non-null object
ERR_ID             0 non-null object
ERR_NAME           20 non-null object
ERR_DESCRIPTION    0 non-null object
ERR_DOWNLOAD       0 non-null object
ERR_TAGS           0 non-null object
ERR_DATA           20 non-null object
ERR_RELATEDW       20 non-null object
DESCRIPTION        20 non-null object
TAGS               20 non-null object
DATA               0 non-null float64
RELATEDW           0 non-null float64
dtypes: bool(5), float64(2), object(15)
memory usage: 2.9+ KB


,ID,NAME,ELEMENT_URL,DOWNLOAD_URL,HAS_ID,HAS_NAME,HAS_DESCRIPTION,HAS_DOWNLOAD,HAS_TAGS,HAS_DATA,...,ERR_NAME,ERR_DESCRIPTION,ERR_DOWNLOAD,ERR_TAGS,ERR_DATA,ERR_RELATEDW,DESCRIPTION,TAGS,DATA,RELATEDW
0,s0005V1962,"The Potato Eaters, 1885",https://vangoghmuseum.nl/en/collection/s0005V1962,https://vangoghmuseum.nl/download/8bc21467-2e7...,True,True,True,True,True,N.A.,...,,None,None,None,N.A.,N.A.,"{\n ""tittle"": ""The Potato Eaters"",\n ""au...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
1,s0019V1962,Garden with Courting Couples: Square Saint-Pie...,https://vangoghmuseum.nl/en/collection/s0019V1962,https://vangoghmuseum.nl/download/12bd07b2-3c2...,True,True,True,True,True,N.A.,...,,None,None,None,N.A.,N.A.,"{\n ""tittle"": ""Garden with Courting Couples...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
2,s0027V1962,"The Langlois Bridge, 1888",https://vangoghmuseum.nl/en/collection/s0027V1962,https://vangoghmuseum.nl/download/c82c2fcd-90e...,True,True,True,True,True,N.A.,...,,None,None,None,N.A.,N.A.,"{\n ""tittle"": ""The Langlois Bridge"",\n ""...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
3,s0029V1962,"The Sower, 1888",https://vangoghmuseum.nl/en/collection/s0029V1962,https://vangoghmuseum.nl/download/61fbedad-1d6...,True,True,True,True,True,N.A.,...,,None,None,None,N.A.,N.A.,"{\n ""tittle"": ""The Sower"",\n ""authorship...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
4,s0031V1962,"Sunflowers, 1889",https://vangoghmuseum.nl/en/collection/s0031V1962,https://vangoghmuseum.nl/download/597e7189-79d...,True,True,True,True,True,N.A.,...,,None,None,None,N.A.,N.A.,"{\n ""tittle"": ""Sunflowers"",\n ""authorshi...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN


In [22]:
# guardando los links en TXT y CSV para backup del proceso y control de calidad
galleryFile = "VanGoghGalleryIndex.csv"
dataFolder = "data"

# actualizando archivo original CSV
# si no existe el directorio
if not os.path.exists(dataFolder):
    
    os.makedirs(dataFolder)

# si existe el directorio
else: 
    
    # guardo el DataFrame segun las ultimas actualizaciones
    paintsFrame.to_csv(os.path.join(os.getcwd(), dataFolder, galleryFile), sep = ",", index = False, encoding = "utf-8", mode = "w")

### Recuperar la desripcion del elemento de la galeria

Con la columna de "ELEMENT_URL", "ID", "HAS_DESCRIPTION" y "ERR_DESCRIPTION" del dataFrame se hacen los siguientes pasos:

- Revisar el enlace en "ELEMENT_URL" si el enlace esta activo se sigue adelante.
- se recupera la seccion donde esta la descripcion del elemento.
- se actualiza el dataframe para mantener la trasabilidad del proceso.
- si la descripcion ya esta, no se repite el trabajo.

se supone que los subfolders de los elementos ya existen.

In [23]:
# recorro el arreglo de enlaces habilitados para extraer la informacion
# creo las carpetas necesarias, si ya existen no las sobreescribo.
links = list(paintsFrame["ELEMENT_URL"])
# en este caso los IDs son los nombres de los folders en local
folders = list(paintsFrame["ID"])
# deberia haber un JSON en esta columna
descriptions = list(paintsFrame["DESCRIPTION"])

# variables de control de calidad del proceso, en este caso en las columnas, "HAS_DESCRIPTION", "ERR_DESCRIPTION"
errList = list()
hasList = list()

# lista donde se guarda la nueva informacion que se recupera de la descripcion y contenido de la obra.
newInfo = list()

# atributos de la busqueda en beatifulsoup
soupTag = "article"
soupAttrs = {"class":"col"}

# formato del diccionario para el JSON de persistencia de la descripcion del elemento
infoDictSchema = {
    "tittle": str(), # nombre o titulo de la obra.
    "authorship": { # diccionario para la autoria de la obra, nombre del autor, lugar y fecha aproximada.
        "name": str(),
        "place": str(),
        "date": str(),
    }, 
    "description": { # diccionario para la descripcion basica de la obra, medio y dimensiones.
        "medium": str(),
        "dimensions": str(),
    },
    "credits": { # diccionario para describir los creditos asociados con la obra.
        "holder": str(),
        "place": str(),
    }, 
    "content": { # diccionario con la descripcion detallada del contenido de la obra y de ser necesario los links asociados
        "text": str(),
        "references": list()
    } 
}
counter = 1

#inicio del ciclo para los enlaces, zip funciona para iterar 2 arreglos al tiempo
for link, folder, des in zip(links, folders, descriptions):
    
    # chequeo si el enlace sirve para sacar la informacion
    try:
        # inicializo uuna variable temporal
        tempInfoDict = copy.deepcopy(infoDictSchema)

        # GET del URL
        linkReq = requests.get(link)

        # si el GET me responde bien con codigo 200
        if linkReq.status_code == 200:
            
            # creo path para cada elemento en la coleccion
            tempPaintFolder = os.path.join(workPath, folder)
            
            # creo el nombre del archivo JSON donde guardo la descripcion
            fileName = "des_" + folder + ".json"
            filePath = os.path.join(tempPaintFolder, fileName)
            
            # si existe la carpeta y no existe el JSON, proceso la informacion de la descripcion
            if os.path.exists(tempPaintFolder) and not os.path.exists(filePath):
                
                # parse del cuerpo del elemento de la coleccion
                linkSoup = BeautifulSoup(linkReq.content, "html.parser")

                # busco todos los elementos de tipo class="button dark-hover"
                downloadSoup = linkSoup.find(soupTag, attrs = soupAttrs)
                
                # recuperando la informacion del titulo
                tempInfoDict["tittle"] = downloadSoup.find("a", attrs = {"class":"text-underline-none", "name":"info"}).text
                
                # recuperando la informacion de la autoria de la obra, nombre del autor, lugar y fecha aproximada.
                tempData = downloadSoup.find("p", attrs = {"class":"text-bold"}).text
                
                # limpiando el texto de caracteres innecesarios
                tempData = re.sub(" \s+", "", tempData)
                tempData = re.sub("\n", "", tempData)
                tempData = tempData.split(",")             
                
                # nombre del autor, lugar de autoria y fecha aproximada
                tempInfoDict["authorship"]["name"] = tempData[0]
                tempInfoDict["authorship"]["place"] = tempData[1]
                tempInfoDict["authorship"]["date"] = tempData[2]
                
                # palabra clave en el esquema de la obra
                keyWord = "Credits"
                
                # recuperando la descripcion de la en un diccionario que contiene medio y dimensiones.
                tempData = downloadSoup.find("p", attrs = {"class":"text-bold"}).findNext("p").text
                tempData = re.sub(" \s+", "", tempData)
                tempData = re.sub("\n", "", tempData)
                tempData = tempData.split(keyWord)
                
                # si encuentro el parrafo con los creditos con la estructura apropiada, dividido por ":"
                if len(tempData) > 1:
                    
                    # tomo el primer elemento
                    tempData = tempData[0]
                    tempData = tempData.split(",")   
                    
                    # guardo en el diccionario el medio y las dimensiones
                    tempInfoDict["description"]["medium"] = tempData[0]
                    tempInfoDict["description"]["dimensions"] = tempData[1]
                    
                # recuperando la descripcion de los creditos asociados con la obra.
                tempData = downloadSoup.find("p", attrs = {"class":"text-bold"}).findNext("p").text
                # limpio y preparo lo que encontre
                tempData = re.sub(" \s+", "", tempData)
                tempData = re.sub("\n", "", tempData)
                tempData = tempData.split(":")
                
                # si encuentro el parrafo con los creditos con la estructura apropiada, dividido por ":"
                if len(tempData) == 2:
                    
                    # tomo el ultimo elemento
                    tempData = tempData[len(tempData)-1]
                    tempData = tempData.split(",")      
                    
                    # guardo en el diccionario el poseedor y locacion de los creditos
                    tempInfoDict["credits"]["holder"] = tempData[0]
                    tempInfoDict["credits"]["place"] = tempData[1]

                # recuperando el contenido de la obra, OJO aqui puede haber enlaces a otros lados!!!!
                tempData = downloadSoup.find("hr", attrs = {"class":"reset-left"}).findNext("p")
                
                # limpio el texto y guardo en el diccionario el texto del contenido
                # si no hay espacios entre signos de puntionacion los pongo
                tempData = re.sub(r"(?<=[.,;:])(?=[^\s])", r" ", tempData.text)
                tempInfoDict["content"]["text"] = tempData
                
                # lo vuelvo a recuperar porque lo altere anteriormente
                tempData = downloadSoup.find("hr", attrs = {"class":"reset-left"}).findNext("p")

                # guardo en el diccionario la lista de enlaces vacia
                tempInfoDict["content"]["references"] = list()

                # lista de posibles enlaces externos
                linkList = list()

                # creo una lista para guardar los links dentro del contenido si es que existen.
                if len(tempData.findAll("a")) > 0:
                    
                    # print("there are " + str(counter) + " links inside the description!!!!")
                    # counter += 1
                    
                    for ref in tempData.findAll("a"):
                        
                        newLink = {"text":ref.text, "link":ref.get("href")}
                        linkList.append(copy.deepcopy(newLink))
                    
                # guardo en el diccionario la lista de enlaces relevantes del texto.
                tempInfoDict["content"]["references"] = copy.deepcopy(linkList)

                # transformo el dict a JSON.
                jsonInfo = json.dumps(copy.deepcopy(tempInfoDict), ensure_ascii = False, indent = 4)
                
                # guardo en un archivo el JSON de la descripcion.
                with open(filePath, "w+", encoding = "utf-8") as file:
                    file.write(jsonInfo)
                    file.close()

                # si no hay errores agrego un mensaje vacio para no danhar el dataframe
                errList.append(None)
                hasList.append(bool(True))
                
            elif os.path.exists(tempPaintFolder) and os.path.exists(filePath):

                jsonInfo = des

                # si no hay errores agrego un mensaje vacio para no danhar el dataframe
                errList.append(None)
                hasList.append(bool(True))           
                    
            # agrego el JSON en una columna del dataframe
            newInfo.append(jsonInfo)
                    
    except Exception as e:
        print("Error in: " + str(e))
        print("Status Code: " + str(linkReq.status_code))
        newInfo.append(copy.deepcopy(infoDictSchema))
        errList.append(e)
        hasList.append(bool(False))

In [24]:
# actualizo el dataFrame para ver si hay errores
paintsFrame["HAS_DESCRIPTION"] = hasList
paintsFrame["ERR_DESCRIPTION"] = errList

# creo una nueva columna para guardar los los diccionarios/JSON dentro del CVS, tambien los guardo en JSON aparte!!!!
paintsFrame["DESCRIPTION"] = newInfo

In [26]:
# guardando los links en TXT y CSV para backup del proceso y control de calidad
galleryFile = "VanGoghGalleryIndex.csv"
dataFolder = "data"

# actualizando archivo original CSV
# si no existe el directorio
if not os.path.exists(dataFolder):
    
    os.makedirs(dataFolder)

# si existe el directorio
else: 
    
    # guardo el DataFrame segun las ultimas actualizaciones
    paintsFrame.to_csv(os.path.join(os.getcwd(), dataFolder, galleryFile), sep = ",", index = False, encoding = "utf-8", mode = "w")

In [25]:
# chequeo como va el dataframe de procesamiento
paintsFrame.info()
paintsFrame.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 22 columns):
ID                 20 non-null object
NAME               20 non-null object
ELEMENT_URL        20 non-null object
DOWNLOAD_URL       20 non-null object
HAS_ID             20 non-null bool
HAS_NAME           20 non-null bool
HAS_DESCRIPTION    20 non-null bool
HAS_DOWNLOAD       20 non-null bool
HAS_TAGS           20 non-null bool
HAS_DATA           20 non-null object
HAS_RELATEDW       20 non-null object
ERR_ID             0 non-null object
ERR_NAME           20 non-null object
ERR_DESCRIPTION    0 non-null object
ERR_DOWNLOAD       0 non-null object
ERR_TAGS           0 non-null float64
ERR_DATA           20 non-null object
ERR_RELATEDW       20 non-null object
DESCRIPTION        20 non-null object
TAGS               20 non-null object
DATA               0 non-null float64
RELATEDW           0 non-null float64
dtypes: bool(5), float64(3), object(14)
memory usage: 2.9+ KB


,ID,NAME,ELEMENT_URL,DOWNLOAD_URL,HAS_ID,HAS_NAME,HAS_DESCRIPTION,HAS_DOWNLOAD,HAS_TAGS,HAS_DATA,...,ERR_NAME,ERR_DESCRIPTION,ERR_DOWNLOAD,ERR_TAGS,ERR_DATA,ERR_RELATEDW,DESCRIPTION,TAGS,DATA,RELATEDW
0,s0005V1962,"The Potato Eaters, 1885",https://vangoghmuseum.nl/en/collection/s0005V1962,https://vangoghmuseum.nl/download/8bc21467-2e7...,True,True,True,True,True,N.A.,...,,None,None,NaN,N.A.,N.A.,"{\n ""tittle"": ""The Potato Eaters"",\n ""au...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
1,s0019V1962,Garden with Courting Couples: Square Saint-Pie...,https://vangoghmuseum.nl/en/collection/s0019V1962,https://vangoghmuseum.nl/download/12bd07b2-3c2...,True,True,True,True,True,N.A.,...,,None,None,NaN,N.A.,N.A.,"{\n ""tittle"": ""Garden with Courting Couples...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
2,s0027V1962,"The Langlois Bridge, 1888",https://vangoghmuseum.nl/en/collection/s0027V1962,https://vangoghmuseum.nl/download/c82c2fcd-90e...,True,True,True,True,True,N.A.,...,,None,None,NaN,N.A.,N.A.,"{\n ""tittle"": ""The Langlois Bridge"",\n ""...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
3,s0029V1962,"The Sower, 1888",https://vangoghmuseum.nl/en/collection/s0029V1962,https://vangoghmuseum.nl/download/61fbedad-1d6...,True,True,True,True,True,N.A.,...,,None,None,NaN,N.A.,N.A.,"{\n ""tittle"": ""The Sower"",\n ""authorship...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
4,s0031V1962,"Sunflowers, 1889",https://vangoghmuseum.nl/en/collection/s0031V1962,https://vangoghmuseum.nl/download/597e7189-79d...,True,True,True,True,True,N.A.,...,,None,None,NaN,N.A.,N.A.,"{\n ""tittle"": ""Sunflowers"",\n ""authorshi...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN


### Recuperar la lista de anotaciones del elemento en la galeria

Con la columna de "ELEMENT_URL", "ID", "TAGS", "HAS_TAGS" y "ERR_TAGS" del dataFrame se hacen los siguientes pasos:

- Revisar el enlace en "ELEMENT_URL" si el enlace esta activo se sigue adelante.
- se recupera la seccion donde esta las anotaciones de busqueda del elemento, esta bajo el texto de "Search in the collection:".
- se actualiza el dataframe para mantener la trasabilidad del proceso.
- si las anotaciones ya estan, no se repite el trabajo.

se supone que los subfolders de los elementos ya existen.

In [32]:
# recorro el arreglo de enlaces habilitados para extraer la informacion
# creo las carpetas necesarias, si ya existen no las sobreescribo.
links = list(paintsFrame["ELEMENT_URL"])
# en este caso los IDs son los nombres de los folders en local
folders = list(paintsFrame["ID"])
# deberia haber un JSON en esta columna
tags = list(paintsFrame["TAGS"])

# variables de control de calidad del proceso, en este caso en las columnas, "HAS_DESCRIPTION", "ERR_DESCRIPTION"
errList = list()
hasList = list()

# lista donde se guarda la nueva informacion que se recupera de la descripcion y contenido de la obra.
newData = list()

# atributos de la busqueda en beatifulsoup
soupTag = "ul"
soupAttrs = {"class":"list-wrapping"}

# formato del diccionario para el JSON de persistencia para las anotaciones del elemento
dataDictSchema = {
    "tags": { # diccionario con la lista detallada de las anotaciones de la obra
        "references": list(), # la lista tiene el link y texto/tag asociado
    } 
}
counter = 1

#inicio del ciclo para los enlaces, zip funciona para iterar 2 arreglos al tiempo
for link, folder, tag in zip(links, folders, tags):
    
    # chequeo si el enlace sirve para sacar la informacion
    try:
        # inicializo uuna variable temporal
        tempInfoDict = copy.deepcopy(dataDictSchema)

        # GET del URL
        linkReq = requests.get(link)

        # si el GET me responde bien con codigo 200
        if linkReq.status_code == 200:
            
            # creo path para cada elemento en la coleccion
            tempPaintFolder = os.path.join(workPath, folder)
            
            # creo el nombre del archivo JSON donde guardo la descripcion
            fileName = "tags_" + folder + ".json"
            filePath = os.path.join(tempPaintFolder, fileName)
            
            # si existe la carpeta y no existe el JSON, proceso la informacion de la descripcion
            if os.path.exists(tempPaintFolder) and not os.path.exists(filePath):
                
                # parse del cuerpo del elemento de la coleccion
                linkSoup = BeautifulSoup(linkReq.content, "html.parser")

                # busco todos los elementos de tipo class="list-wrapping"
                tagsSoup = linkSoup.find(soupTag, attrs = soupAttrs)
                
                # saco la informacion que necesito de los tags de busqueda
                tempData = tagsSoup.findAll("a", attrs = {"class":"button outline neutral text-titlecase"})
                                
                # lista de posibles enlaces externos
                linkList = list()

                # recopilo la lista dentro del elemento encontrado
                for ref in tempData:#.findAll("a", attrs = {"class":"button outline neutral text-titlecase"}):

                    newLink = {"text":ref.text, "link":ref.get("href")} 
                    linkList.append(copy.deepcopy(newLink))

                # guardo en el diccionario la lista de enlaces relevantes de las anotaciones.
                tempInfoDict["tags"]["references"] = copy.deepcopy(linkList)

                # transformo el dict a JSON.
                jsonData = json.dumps(copy.deepcopy(tempInfoDict), ensure_ascii = False, indent = 4)
                
                # guardo en un archivo el JSON de las anotaciones.
                with open(filePath, "w+", encoding = "utf-8") as file:
                    file.write(jsonData)
                    file.close()

                # si no hay errores agrego un mensaje vacio para no danhar el dataframe
                errList.append(None)
                hasList.append(bool(True))
            
            # si ya existe lo que voy a escribir
            elif os.path.exists(tempPaintFolder) and os.path.exists(filePath):
                
                # solo lo tomo del dataframe y lo vuelvo a cargar
                jsonData = tag

                # si no hay errores agrego un mensaje vacio para no danhar el dataframe
                errList.append(None)
                hasList.append(bool(True))           
                    
            # agrego el JSON en una columna del dataframe
            newData.append(jsonData)
                    
    except Exception as e:
        print("Error in: " + str(e))
        print("Status Code: " + str(linkReq.status_code))
        newData.append(copy.deepcopy(dataDictSchema))
        errList.append(e)
        hasList.append(bool(False))

In [29]:
# actualizo el dataFrame para ver si hay errores
paintsFrame["HAS_TAGS"] = hasList
paintsFrame["ERR_TAGS"] = errList

# creo una nueva columna para guardar las anotaciones en JSON dentro del CVS, tambien guardo JSON aparte!!!!
paintsFrame["TAGS"] = newData

In [45]:
# chequeo como va el dataframe de procesamiento
paintsFrame.info()
paintsFrame.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 22 columns):
ID                 20 non-null object
NAME               20 non-null object
ELEMENT_URL        20 non-null object
DOWNLOAD_URL       20 non-null object
HAS_ID             20 non-null bool
HAS_NAME           20 non-null bool
HAS_DESCRIPTION    20 non-null bool
HAS_DOWNLOAD       20 non-null bool
HAS_TAGS           20 non-null bool
HAS_DATA           20 non-null object
HAS_RELATEDW       20 non-null object
ERR_ID             0 non-null object
ERR_NAME           0 non-null object
ERR_DESCRIPTION    0 non-null object
ERR_DOWNLOAD       0 non-null object
ERR_TAGS           0 non-null object
ERR_DATA           20 non-null object
ERR_RELATEDW       20 non-null object
DESCRIPTION        20 non-null object
TAGS               20 non-null object
DATA               0 non-null float64
RELATEDW           0 non-null float64
dtypes: bool(5), float64(2), object(15)
memory usage: 2.9+ KB


,ID,NAME,ELEMENT_URL,DOWNLOAD_URL,HAS_ID,HAS_NAME,HAS_DESCRIPTION,HAS_DOWNLOAD,HAS_TAGS,HAS_DATA,...,ERR_NAME,ERR_DESCRIPTION,ERR_DOWNLOAD,ERR_TAGS,ERR_DATA,ERR_RELATEDW,DESCRIPTION,TAGS,DATA,RELATEDW
0,s0005V1962,"The Potato Eaters, 1885",https://vangoghmuseum.nl/en/collection/s0005V1962,https://vangoghmuseum.nl/download/8bc21467-2e7...,True,True,True,True,True,N.A.,...,None,None,None,None,N.A.,N.A.,"{\n ""tittle"": ""The Potato Eaters"",\n ""au...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
1,s0019V1962,Garden with Courting Couples: Square Saint-Pie...,https://vangoghmuseum.nl/en/collection/s0019V1962,https://vangoghmuseum.nl/download/12bd07b2-3c2...,True,True,True,True,True,N.A.,...,None,None,None,None,N.A.,N.A.,"{\n ""tittle"": ""Garden with Courting Couples...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
2,s0027V1962,"The Langlois Bridge, 1888",https://vangoghmuseum.nl/en/collection/s0027V1962,https://vangoghmuseum.nl/download/c82c2fcd-90e...,True,True,True,True,True,N.A.,...,None,None,None,None,N.A.,N.A.,"{\n ""tittle"": ""The Langlois Bridge"",\n ""...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
3,s0029V1962,"The Sower, 1888",https://vangoghmuseum.nl/en/collection/s0029V1962,https://vangoghmuseum.nl/download/61fbedad-1d6...,True,True,True,True,True,N.A.,...,None,None,None,None,N.A.,N.A.,"{\n ""tittle"": ""The Sower"",\n ""authorship...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN
4,s0031V1962,"Sunflowers, 1889",https://vangoghmuseum.nl/en/collection/s0031V1962,https://vangoghmuseum.nl/download/597e7189-79d...,True,True,True,True,True,N.A.,...,None,None,None,None,N.A.,N.A.,"{\n ""tittle"": ""Sunflowers"",\n ""authorshi...","{\n ""tags"": {\n ""references"": [\n ...",NaN,NaN


In [44]:
# guardando los links en TXT y CSV para backup del proceso y control de calidad
galleryFile = "VanGoghGalleryIndex.csv"
dataFolder = "data"

# actualizando archivo original CSV
# si no existe el directorio
if not os.path.exists(dataFolder):
    
    os.makedirs(dataFolder)

# si existe el directorio
else: 
    
    # guardo el DataFrame segun las ultimas actualizaciones
    paintsFrame.to_csv(os.path.join(os.getcwd(), dataFolder, galleryFile), sep = ",", index = False, encoding = "utf-8", mode = "w")

### Recuperar los datos de coleccion del elemento de la galeria

Con la columna de "ELEMENT_URL", "ID", "DATA", "HAS_DATA" y "ERR_DATA" del dataFrame se hacen los siguientes pasos:

- Revisar el enlace en "ELEMENT_URL" si el enlace esta activo se sigue adelante.
- se recupera la seccion donde esta los datos de archivo del elemento.
- se actualiza el dataframe para mantener la trasabilidad del proceso.
- si la descripcion ya esta, no se repite el trabajo.

se supone que los subfolders de los elementos ya existen.

In [100]:
# recorro el arreglo de enlaces habilitados para extraer la informacion
# creo las carpetas necesarias, si ya existen no las sobreescribo.
links = list(paintsFrame["ELEMENT_URL"])
# en este caso los IDs son los nombres de los folders en local
folders = list(paintsFrame["ID"])
# deberia haber un JSON en esta columna
objects = list(paintsFrame["DATA"])

# variables de control de calidad del proceso, en este caso en las columnas, "HAS_DESCRIPTION", "ERR_DESCRIPTION"
errList = list()
hasList = list()

# lista donde se guarda la nueva informacion que se recupera de la descripcion y contenido de la obra.
newData = list()

# atributos de la busqueda en beatifulsoup
soupTag = "dl"
soupAttrs = {"class":"list-table compact"}

# formato del diccionario para el JSON de persistencia de la descripcion del elemento
dataDictSchema = {
    "F_number": str(), # numero de referencia F de la obra.
    "JH_number": str(), # numero de referencia JH de la obra.
    "obj_number": str(), # numero de referencia del objeto de la obra.
    "dimensions": { # diccionario con las dimensiones basicas de la obra, canvas y marco.
        "canvas": str(),
        "frame": str(),
    },
    "credits": { # diccionario para los creditos asociados con la obra.
        "holder": str(),
        "place": str(),
    },
}
counter = 1

#inicio del ciclo para los enlaces, zip funciona para iterar 2 arreglos al tiempo
for link, folder, obj in zip(links, folders, objects):
    
    # chequeo si el enlace sirve para sacar la informacion
    try:
        # inicializo uuna variable temporal
        tempInfoDict = copy.deepcopy(dataDictSchema)

        # GET del URL
        linkReq = requests.get(link)

        # si el GET me responde bien con codigo 200
        if linkReq.status_code == 200:
            
            # creo path para cada elemento en la coleccion
            tempPaintFolder = os.path.join(workPath, folder)
            
            # creo el nombre del archivo JSON donde guardo la descripcion
            fileName = "data_" + folder + ".json"
            filePath = os.path.join(tempPaintFolder, fileName)
            
            # si existe la carpeta y no existe el JSON, proceso la informacion de la descripcion
            if os.path.exists(tempPaintFolder) and not os.path.exists(filePath):
                
                # parse del cuerpo del elemento de la coleccion
                linkSoup = BeautifulSoup(linkReq.content, "html.parser")

                # busco todos los elementos de tipo class="button dark-hover"
                dataSoup = linkSoup.find(soupTag, attrs = soupAttrs)
                
                # encuentro los titulos de las celdas
                tempDataCol1 = dataSoup.findAll("dt", attrs = {"class":"text-titlecase"})
                
                # encuentro los datos no-vacios de las celdas
                tempDataCol2 = dataSoup.findAll("dd", string = re.compile("^(?!\s*$).+"))
                
                if len(tempDataCol1) == len(tempDataCol2):
                    
                    for data1, data2 in zip(tempDataCol1, tempDataCol2):
                        
                        # si encuentro el numero F
                        if data1.text == "F-number":
                            
                            # limpio el dato de ser necesario y lo pongo en el diccionario
                            tempInfoDict["F_number"] = data2.text.strip(" ")
                            
                        # si encuentro el numero JH
                        if data1.text == "JH_number":
                            
                            # limpio el dato de ser necesario y lo pongo en el diccionario
                            tempInfoDict["JH_number"] = data2.text.strip(" ")

                        # si encuentro el numero JH
                        if data1.text == "Object number":
                            
                            # limpio el dato de ser necesario y lo pongo en el diccionario
                            tempInfoDict["obj_number"] = data2.text.strip(" ")
                        
                        # si encuentro los creditos de la obra
                        if "Credits" in data1.text:
                            
                            tempData = data2.text.split(",")
                            
                            if len(tempData) == 2:
                                
                                tempInfoDict["dimensions"]["holder"] = tempData[0]
                                tempInfoDict["dimensions"]["place"] = tempData[1]
                        
                        # si encuentro las dimensiones de la obra
                        if data1.text == "Dimensions":
                            
                            tempData = data2.text.split(",")
                            
                            if len(tempData) == 2:
                                
                                tempInfoDict["dimensions"]["canvas"] = tempData[0]
                                tempInfoDict["dimensions"]["frame"] = tempData[1]
                            
                            else:
                                
                                tempInfoDict["dimensions"]["canvas"] = tempData[0]
                                tempInfoDict["dimensions"]["frame"] = tempData[0]                               

                # transformo el dict a JSON.
                jsonInfo = json.dumps(copy.deepcopy(tempInfoDict), ensure_ascii = False, indent = 4)
                
                # guardo en un archivo el JSON de la descripcion.
                with open(filePath, "w+", encoding = "utf-8") as file:
                    file.write(jsonInfo)
                    file.close()

                # si no hay errores agrego un mensaje vacio para no danhar el dataframe
                errList.append(None)
                hasList.append(bool(True))
                
            elif os.path.exists(tempPaintFolder) and os.path.exists(filePath):

                jsonInfo = obj

                # si no hay errores agrego un mensaje vacio para no danhar el dataframe
                errList.append(None)
                hasList.append(bool(True))           
                    
            # agrego el JSON en una columna del dataframe
            newData.append(jsonInfo)
                    
    except Exception as e:
        print("Error in: " + str(e))
        print("Status Code: " + str(linkReq.status_code))
        newData.append(copy.deepcopy(dataDictSchema))
        errList.append(e)
        hasList.append(bool(False))

In [101]:
# actualizo el dataFrame para ver si hay errores
paintsFrame["HAS_DATA"] = hasList
paintsFrame["ERR_DATA"] = errList

# creo una nueva columna para guardar los diccionarios/JSON dentro del CVS, tambien los guardo en JSON aparte!!!!
paintsFrame["DATA"] = newData

In [102]:
# chequeo como va el dataframe de procesamiento
paintsFrame.info()
paintsFrame.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 22 columns):
ID                 20 non-null object
NAME               20 non-null object
ELEMENT_URL        20 non-null object
DOWNLOAD_URL       20 non-null object
HAS_ID             20 non-null bool
HAS_NAME           20 non-null bool
HAS_DESCRIPTION    20 non-null bool
HAS_DOWNLOAD       20 non-null bool
HAS_TAGS           20 non-null bool
HAS_DATA           20 non-null bool
HAS_RELATEDW       20 non-null object
ERR_ID             0 non-null object
ERR_NAME           0 non-null object
ERR_DESCRIPTION    0 non-null object
ERR_DOWNLOAD       0 non-null object
ERR_TAGS           0 non-null object
ERR_DATA           0 non-null object
ERR_RELATEDW       20 non-null object
DESCRIPTION        20 non-null object
TAGS               20 non-null object
DATA               20 non-null object
RELATEDW           0 non-null float64
dtypes: bool(6), float64(1), object(15)
memory usage: 2.7+ KB


,ID,NAME,ELEMENT_URL,DOWNLOAD_URL,HAS_ID,HAS_NAME,HAS_DESCRIPTION,HAS_DOWNLOAD,HAS_TAGS,HAS_DATA,...,ERR_NAME,ERR_DESCRIPTION,ERR_DOWNLOAD,ERR_TAGS,ERR_DATA,ERR_RELATEDW,DESCRIPTION,TAGS,DATA,RELATEDW
0,s0005V1962,"The Potato Eaters, 1885",https://vangoghmuseum.nl/en/collection/s0005V1962,https://vangoghmuseum.nl/download/8bc21467-2e7...,True,True,True,True,True,True,...,None,None,None,None,None,N.A.,"{\n ""tittle"": ""The Potato Eaters"",\n ""au...","{\n ""tags"": {\n ""references"": [\n ...","{\n ""F_number"": ""F0082"",\n ""JH_number"": ...",NaN
1,s0019V1962,Garden with Courting Couples: Square Saint-Pie...,https://vangoghmuseum.nl/en/collection/s0019V1962,https://vangoghmuseum.nl/download/12bd07b2-3c2...,True,True,True,True,True,True,...,None,None,None,None,None,N.A.,"{\n ""tittle"": ""Garden with Courting Couples...","{\n ""tags"": {\n ""references"": [\n ...","{\n ""F_number"": ""F0314"",\n ""JH_number"": ...",NaN
2,s0027V1962,"The Langlois Bridge, 1888",https://vangoghmuseum.nl/en/collection/s0027V1962,https://vangoghmuseum.nl/download/c82c2fcd-90e...,True,True,True,True,True,True,...,None,None,None,None,None,N.A.,"{\n ""tittle"": ""The Langlois Bridge"",\n ""...","{\n ""tags"": {\n ""references"": [\n ...","{\n ""F_number"": ""F0400"",\n ""JH_number"": ...",NaN
3,s0029V1962,"The Sower, 1888",https://vangoghmuseum.nl/en/collection/s0029V1962,https://vangoghmuseum.nl/download/61fbedad-1d6...,True,True,True,True,True,True,...,None,None,None,None,None,N.A.,"{\n ""tittle"": ""The Sower"",\n ""authorship...","{\n ""tags"": {\n ""references"": [\n ...","{\n ""F_number"": ""F0451"",\n ""JH_number"": ...",NaN
4,s0031V1962,"Sunflowers, 1889",https://vangoghmuseum.nl/en/collection/s0031V1962,https://vangoghmuseum.nl/download/597e7189-79d...,True,True,True,True,True,True,...,None,None,None,None,None,N.A.,"{\n ""tittle"": ""Sunflowers"",\n ""authorshi...","{\n ""tags"": {\n ""references"": [\n ...","{\n ""F_number"": ""F0458"",\n ""JH_number"": ...",NaN


In [103]:
# guardando los links en TXT y CSV para backup del proceso y control de calidad
galleryFile = "VanGoghGalleryIndex.csv"
dataFolder = "data"

# actualizando archivo original CSV
# si no existe el directorio
if not os.path.exists(dataFolder):
    
    os.makedirs(dataFolder)

# si existe el directorio
else: 
    
    # guardo el DataFrame segun las ultimas actualizaciones
    paintsFrame.to_csv(os.path.join(os.getcwd(), dataFolder, galleryFile), sep = ",", index = False, encoding = "utf-8", mode = "w")

In [2]:
# lista de columnas para trabajar el DataFrame de la galeria, sirve para manejar la calidad del proceso
# conocer los errores y ver que datos faltan por cada elemento de la galeria
soupCol = [
    "ID", # identificador unico de la galeria, tambien es el nombre del folder dentro del directorio local
    "NAME", # nombre del elemento en la galeria
    "ELEMENT_URL", # enlace del elemento recuperado del ScrapyWEB
    "DOWNLOAD_URL", # enlace de la imagen dentro del elemento de la galeria

    "HAS_ID", # booleano que identifica si se tiene un folder local del elemento
    "HAS_NAME", # booleano que identifica si se tiene el nombre del elemento
    
    "HAS_DESCRIPTION", # booleano que identifica si se tiene la seccion de descripcion en el HTML del elemento
    "HAS_DOWNLOAD", # booleano que identifica si se tiene la seccion de enlace de descarga en el HTML del elemento
    "HAS_TAGS", # booleano que identifica si se tiene la seccion de tags de busqueda en el HTML del elemento
    "HAS_DATA", # booleano que identifica si se tiene la seccion de datos de archivo en el HTML del elemento
    "HAS_RELATEDW", # booleano que identifica si se tiene la seccion de trabajo relacionado en el HTML del elemento
    
    "ERR_ID", # si no se puede crear la carpeta con el ID se guarda el error aca
    "ERR_NAME", # si no se obtiene el nombre se guarda el error aca
    
    "ERR_DESCRIPTION", # si no se obtiene la descripcion se guarda el error aca 
    "ERR_DOWNLOAD", # si no se obtiene el enlace de descarga se guarda el error aca 
    "ERR_TAGS", # si no se obtiene los tags de busqueda se guarda el error aca 
    "ERR_DATA", # si no se obtiene los datos de archivo se guarda el error aca 
    "ERR_RELATEDW", # si no se obtiene el trabajo relacionado se guarda el error aca 
    
    "DESCRIPTION", # aqui guardo el JSON con la informacion de la descripcion
    "TAGS", # aqui guardo el JSON con la informacion de los tags de busqueda
    "DATA", # aqui guardo el JSON con la informacion de los datos de coleccion
    "RELATEDW", # aqui guardo el JSON con la informacion del trabajo relacionado con cada una de las obras
]

In [ ]:

totalpinturas=1443
temporal=199
pagina='https://vangoghmuseum.nl/en/collection/s'
a=np.arange(temporal)
lista=[]
for i in a:
    lista.append(pagina+str((i+1)).zfill(4)+'V1962')
    URL = lista[i]

    def iterate_over_children(soup, texts):
        for child in soup:
            if child.string == None:
                iterate_over_children(child, texts)
            else:
                if child.string != '\n':
                    texts.append(child.string)

    # Realizamos la petición a la web
    req = requests.get(URL)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    status_code = req.status_code
    if status_code == 200:

        # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
        html = BeautifulSoup(req.text, "html.parser")

        # Obtenemos todos los divs donde están las entradas
        #entradas = html.find_all('div', {'class': 'page-unit'})
        article = html.find('section')
        texts = []
        iterate_over_children(article, texts)
        texts_cleaned = [] 
        for text in texts:
            cleaned_text = text.replace("\n", "").strip()
            texts_cleaned.append(cleaned_text)

    #Daniela Oject Data
        # Obtenemos todos los divs donde están las entradas
        entradas = html.find_all('div', {'class': 'download-buttons-container'})


        # Recorremos todas las entradas para extraer el título, autor y fecha
        for i, entrada in enumerate(entradas):
            # Con el método "getText()" no nos devuelve el HTML
            hola= entrada.find('a').get('href') 
            texts_cleaned.append(hola)
            
        #links del parrafo (puede ir a las cartas)
        link_p=[tag['href'] for tag in html.select('p a[href]')]
        #links de search in the collection
        link_c=[tag['href'] for tag in html.select('aside a[href]')]
            
        
        #print(texts_cleaned)
        os.mkdir(texts_cleaned[0])
        #print (texts_cleaned)
        # Recorremos todas las entradas para extraer el título, autor y fecha
        save_path =texts_cleaned[0]

        f=open(texts_cleaned[0]+'.txt', 'w')

        completeName = os.path.join(save_path,texts_cleaned[0]+'.txt')    
        
        f.close()
        remove(texts_cleaned[0]+'.txt')

        file1 = open(completeName, "w")
        
        tamanio = len(texts_cleaned)
        for i in range(0,len(texts_cleaned)):
            file1.write(texts_cleaned[i]+'\n')
            
        for i in range(0,len(link_p)):
            file1.write(link_p[i]+'\n')
        for i in range(0,len(link_c)):
            file1.write(link_c[i]+'\n')
        file1.close()
      
        print('Beginning file download with wget module')

        url = 'https://vangoghmuseum.nl'+texts_cleaned[tamanio-1]
        wget.download(url, texts_cleaned[0]+'\ ' +texts_cleaned[0]+'.jpg')

        # guardar-paginaweb.py

        respuesta = urllib.request.urlopen(URL)
        contenidoWeb = respuesta.read()

        pag = open(texts_cleaned[0]+'.html', 'wb')
        completeName2 = os.path.join(save_path,texts_cleaned[0]+'.html')    
        pag.close()
        file2 = open(completeName2, "wb")
        file2.write(contenidoWeb)
        file2.close()
        
        remove(texts_cleaned[0]+'.html')
    else:
        print ("Status Code %d" % status_code)


In [ ]:
pag.close()

In [ ]:
page = html.find('p').getText()
print(page)

In [ ]:

print('Beginning file download with wget module')

url = 'https://vangoghmuseum.nl/download/45214344-2680-447a-92d3-e34667866f59.jpg'
wget.download(url, 'Congregation Leaving the Reformed Church in Nuenen\daniela.jpg')


In [ ]:
file1.close()

In [ ]:


URL = "https://vangoghmuseum.nl/en/collection/s0199V1962"

def iterate_over_children(soup, texts):
    for child in soup:
        if child.string == None:
            iterate_over_children(child, texts)
        else:
            if child.string != '\n':
                texts.append(child.string)

# Realizamos la petición a la web
req = requests.get(URL)

# Comprobamos que la petición nos devuelve un Status Code = 200
status_code = req.status_code
if status_code == 200:

    # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
    html = BeautifulSoup(req.text, "html.parser")

    # Obtenemos todos los divs donde están las entradas
    #entradas = html.find_all('div', {'class': 'page-unit'})
    article = html.find('section')
    texts = []
    iterate_over_children(article, texts)
    texts_cleaned = [] 
    for text in texts:
        cleaned_text = text.replace("\n", "").strip()
        texts_cleaned.append(cleaned_text)
  #  print(texts_cleaned)
       
#Daniela Oject Data
    # Obtenemos todos los divs donde están las entradas
    entradas = html.find_all('div', {'class': 'download-buttons-container'})
    

    # Recorremos todas las entradas para extraer el título, autor y fecha
    for i, entrada in enumerate(entradas):
        # Con el método "getText()" no nos devuelve el HTML
        hola= entrada.find('a').get('href') 
        texts_cleaned.append(hola)
    print (texts_cleaned)
    # Recorremos todas las entradas para extraer el título, autor y fecha

#url_imagen = "https://golang.org/doc/gopher/appenginegophercolor.jpg" # El link de la imagen
#nombre_local_imagen = "go.jpg" # El nombre con el que queremos guardarla
#imagen = requests.get(url_imagen).content
#with open(nombre_local_imagen, 'wb') as handler:
#	handler.write(imagen)

else:
    print ("Status Code %d" % status_code)
    



In [ ]:
os.mkdir('Comedores de Patatas')

In [ ]:

save_path ='Comedores de Patatas'

f = open('daniela6.txt', 'w')

completeName = os.path.join(save_path,'daniela6.txt')         

file1 = open(completeName, "w")

file1.write("daniela carcamo")
file1.close()

# remove(completeName)


In [ ]:
# abre-paginaweb.py
url = 'https://www.vangoghmuseum.nl/en/collection/s0005V1962?v=1'

respuesta = urllib.request.urlopen(url)
contenidoWeb = respuesta.read()

print(contenidoWeb[0:300])

In [ ]:
# guardar-paginaweb.py

url = 'https://www.vangoghmuseum.nl/en/collection/s0005V1962?v=1'

respuesta = urllib.request.urlopen(url)
contenidoWeb = respuesta.read()

f = open('patatas.html', 'wb')
f.write(contenidoWeb)
f.close

In [ ]:
print (respuesta)